<a href="https://colab.research.google.com/github/telmavcosta/data_processing/blob/main/spark_streaming/challenges/rep_final_challenges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up PySpark

In [1]:
%pip install pyspark

# Context
Message events are coming from platform message broker (kafka, pubsub, kinesis...).
You need to process the data according to the requirements.

Message schema:
- timestamp
- value
- event_type
- message_id
- country_id
- user_id



# Challenge 1

Step 1
- Change exising producer
	- Change parquet location to "/content/lake/bronze/messages/data"
	- Add checkpoint (/content/lake/bronze/messages/checkpoint)
	- Delete /content/lake/bronze/messages and reprocess data
	- For reprocessing, run the streaming for at least 1 minute, then stop it

Step 2
- Implement new stream job to read from messages in bronze layer and split result in two locations
	- "messages_corrupted"
		- logic: event_status is null, empty or equal to "NONE"
    - extra logic: add country name by joining message with countries dataset
		- partition by "date" -extract it from timestamp
		- location: /content/lake/silver/messages_corrupted/data

	- "messages"
		- logic: not corrupted data
		- extra logic: add country name by joining message with countries dataset
		- partition by "date" -extract it from timestamp
		- location: /content/lake/silver/messages/data

	- technical requirements
		- add checkpint (choose location)
		- use StructSchema
		- Set trigger interval to 5 seconds
		- run streaming for at least 20 seconds, then stop it

	- alternatives
		- implementing single streaming job with foreach/- foreachBatch logic to write into two locations
		- implementing two streaming jobs, one for messages and another for messages_corrupted
		- (paying attention on the paths and checkpoints)


  - Check results:
    - results from messages in bronze layer should match with the sum of messages+messages_corrupted in the silver layer

In [2]:
%pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.5 MB/s eta 0:00:00


# Producer

In [3]:
import pyspark.sql.functions as F
from pyspark.sql import DataFrame
from faker import Faker
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Test streaming').getOrCreate()
sc = spark.sparkContext

fake = Faker()
messages = [fake.uuid4() for _ in range(50)]

def enrich_data(df, messages=messages):
  fake = Faker()
  new_columns = {
      'event_type': F.lit(fake.random_element(elements=('OPEN', 'RECEIVED', 'SENT', 'CREATED', 'CLICKED', '', 'NONE'))),
      'message_id': F.lit(fake.random_element(elements=messages)),
      'channel': F.lit(fake.random_element(elements=('CHAT', 'EMAIL', 'SMS', 'PUSH', 'OTHER'))),
      'country_id': F.lit(fake.random_int(min=2000, max=2015)),
      'user_id': F.lit(fake.random_int(min=1000, max=1050)),
  }
  df = df.withColumns(new_columns)
  return df

def insert_messages(df: DataFrame, batch_id):
  enrich = enrich_data(df)
  enrich.write.mode("append").format("parquet").save("content/lake/bronze/messages/data")

# read stream
df_stream = spark.readStream.format("rate").option("rowsPerSecond", 1).load()

# write stream
query = (df_stream.writeStream
.option('checkpointLocation', 'content/lake/bronze/messages/checkpoint')
.outputMode('append')
.trigger(processingTime='1 seconds')
.foreachBatch(insert_messages)
.start()
)

query.awaitTermination(60)


False

In [16]:
query.stop()

ERROR:py4j.clientserver:There was an exception while executing the Python Proxy on the Python Side.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/py4j/clientserver.py", line 617, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pyspark/sql/utils.py", line 120, in call
    raise e
  File "/usr/local/lib/python3.11/dist-packages/pyspark/sql/utils.py", line 117, in call
    self.func(DataFrame(jdf, wrapped_session_jdf), batch_id)
  File "/tmp/ipython-input-3-2779494543.py", line 26, in insert_messages
    enrich.write.mode("append").format("parquet").save("content/lake/bronze/messages/data")
  File "/usr/local/lib/python3.11/dist-packages/pyspark/sql/readwriter.py", line 1463, in save
    self._jwrite.save(path)
  File "/usr/local/lib/python3.11/dist-packages/py4j/java_gateway.py", line 1322, in __call__
    return

In [24]:
df = spark.read.format("parquet").load("content/lake/bronze/messages/data")
df.show()
df.count()

+--------------------+-----+----------+--------------------+-------+----------+-------+
|           timestamp|value|event_type|          message_id|channel|country_id|user_id|
+--------------------+-----+----------+--------------------+-------+----------+-------+
|2025-07-16 14:56:...|  107|      NONE|41f4d3e9-7f02-417...|  EMAIL|      2012|   1021|
|2025-07-16 14:56:...|  109|      NONE|41f4d3e9-7f02-417...|  EMAIL|      2012|   1021|
|2025-07-16 14:56:...|  111|      NONE|41f4d3e9-7f02-417...|  EMAIL|      2012|   1021|
|2025-07-16 14:56:...|  113|      NONE|41f4d3e9-7f02-417...|  EMAIL|      2012|   1021|
|2025-07-16 14:56:...|  115|      NONE|41f4d3e9-7f02-417...|  EMAIL|      2012|   1021|
|2025-07-16 14:56:...|  117|      NONE|41f4d3e9-7f02-417...|  EMAIL|      2012|   1021|
|2025-07-16 14:56:...|  119|      NONE|41f4d3e9-7f02-417...|  EMAIL|      2012|   1021|
|2025-07-16 14:56:...|  108|      NONE|41f4d3e9-7f02-417...|  EMAIL|      2012|   1021|
|2025-07-16 14:56:...|  110|    

304

In [ ]:
#query.isActive

False

# Additional datasets

In [4]:
countries = [
    {"country_id": 2000, "country": "Brazil"},
    {"country_id": 2001, "country": "Portugal"},
    {"country_id": 2002, "country": "Spain"},
    {"country_id": 2003, "country": "Germany"},
    {"country_id": 2004, "country": "France"},
    {"country_id": 2005, "country": "Italy"},
    {"country_id": 2006, "country": "United Kingdom"},
    {"country_id": 2007, "country": "United States"},
    {"country_id": 2008, "country": "Canada"},
    {"country_id": 2009, "country": "Australia"},
    {"country_id": 2010, "country": "Japan"},
    {"country_id": 2011, "country": "China"},
    {"country_id": 2012, "country": "India"},
    {"country_id": 2013, "country": "South Korea"},
    {"country_id": 2014, "country": "Russia"},
    {"country_id": 2015, "country": "Argentina"}
]

countries = spark.createDataFrame(countries)

# Streaming Messages x Messages Corrupted

In [ ]:
!rm -rf lake/silver/*
!rm -rf "{'"content/lake/*

In [5]:
# TODO

from pyspark.sql.types import *

message_schema = StructType([StructField('timestamp', TimestampType(), True),
                            StructField('value', LongType(), True),
                            StructField('event_type', StringType(), True),
                            StructField('message_id', StringType(), True),
                            StructField('channel', StringType(), True),
                            StructField('country_id', IntegerType(), True),
                            StructField('user_id', IntegerType(), True)])

df_bronze = spark.readStream.format("parquet").schema(message_schema).load("content/lake/bronze/messages/data")


enriched_df = df_bronze.join(countries, on="country_id", how="left").withColumn("country", F.col("country")).withColumn("date", F.to_date(F.col("timestamp")))

# Split into corrupted and valid
is_corrupted = (F.col("event_type").isNull() | (F.trim(F.col("event_type")) == "") | (F.trim(F.col("event_type")) == "NONE"))

corrupted_messages_df = enriched_df.filter(is_corrupted)
valid_messages_df = enriched_df.filter(~is_corrupted)


corrupted_query = corrupted_messages_df.writeStream \
    .format("parquet") \
    .outputMode("append") \
    .option("checkpointLocation", "content/lake/silver/messages_corrupted/checkpoint") \
    .option("path","content/lake/silver/messages_corrupted/data") \
    .partitionBy("date") \
    .trigger(processingTime="5 seconds") \
    .start()

valid_query = valid_messages_df.writeStream \
    .format("parquet") \
    .outputMode("append") \
    .option("checkpointLocation", "content/lake/silver/messages/checkpoint") \
    .option("path","content/lake/silver/messages/data") \
    .partitionBy("date") \
    .trigger(processingTime="5 seconds") \
    .start()

corrupted_query.awaitTermination(20)
valid_query.awaitTermination(20)


False

In [11]:
df_corrupted = spark.read.format("parquet").load("content/lake/silver/messages_corrupted/data")
df_corrupted.show()
df_corrupted.count()

+----------+--------------------+-----+----------+--------------------+-------+-------+---------+----------+
|country_id|           timestamp|value|event_type|          message_id|channel|user_id|  country|      date|
+----------+--------------------+-----+----------+--------------------+-------+-------+---------+----------+
|      2009|2025-07-16 14:54:...|   24|      NONE|6490f409-e827-43a...|  EMAIL|   1009|Australia|2025-07-16|
|      2009|2025-07-16 14:55:...|   45|      NONE|4da9c809-e448-495...|  EMAIL|   1006|Australia|2025-07-16|
|      2009|2025-07-16 14:54:...|   23|      NONE|f4e1f692-8545-4be...|   PUSH|   1035|Australia|2025-07-16|
|      2009|2025-07-16 14:54:...|   17|          |c75aaf6f-b8fb-496...|   CHAT|   1033|Australia|2025-07-16|
|      2010|2025-07-16 14:55:...|   73|      NONE|c899f480-44c6-421...|  EMAIL|   1012|    Japan|2025-07-16|
|      2010|2025-07-16 14:55:...|   77|      NONE|24240b8f-4ab6-4cb...|  OTHER|   1022|    Japan|2025-07-16|
|      2012|2025-07

84

In [27]:
df_valid = spark.read.format("parquet").load("content/lake/silver/messages/data")
df_valid.show()
df_valid.count()

+----------+--------------------+-----+----------+--------------------+-------+-------+-------------+----------+
|country_id|           timestamp|value|event_type|          message_id|channel|user_id|      country|      date|
+----------+--------------------+-----+----------+--------------------+-------+-------+-------------+----------+
|      2000|2025-07-16 14:55:...|   79|   CREATED|41f4d3e9-7f02-417...|  EMAIL|   1011|       Brazil|2025-07-16|
|      2000|2025-07-16 14:55:...|   57|      OPEN|9cf219d2-c0c0-476...|   CHAT|   1004|       Brazil|2025-07-16|
|      2001|2025-07-16 14:55:...|   53|   CLICKED|407f8793-8033-47e...|  OTHER|   1026|     Portugal|2025-07-16|
|      2001|2025-07-16 14:55:...|   36|   CREATED|ff8642f1-4aa6-4f9...|   CHAT|   1036|     Portugal|2025-07-16|
|      2001|2025-07-16 14:56:...|   89|  RECEIVED|e49c1299-a224-408...|    SMS|   1049|     Portugal|2025-07-16|
|      2001|2025-07-16 14:55:...|   68|   CREATED|2df8fff6-c758-43e...|    SMS|   1033|     Port

68

In [28]:
corrupted_query.stop()
valid_query.stop()

## Checking data

In [25]:
# TODO (count messages da silver (corrupted e not corrupted) = mensagens bronze)
df = spark.read.format("parquet").load("content/lake/bronze/messages/data")
df_corrupted = spark.read.format("parquet").load("content/lake/silver/messages_corrupted/data")
df_valid = spark.read.format("parquet").load("content/lake/silver/messages/data")

df_valid.count()
df_corrupted.count()
df.count()

df_valid.count() + df_corrupted.count() == df.count()

False

# Challenge 2

- Run business report
- But first, there is a bug in the system which is causing some duplicated messages, we need to exclude these lines from the report

- removing duplicates logic:
  - Identify possible duplicates on message_id, event_type and channel
  - in case of duplicates, consider only the first message (occurrence by timestamp)
  - Ex:
    In table below, the correct message to consider is the second line

```
    message_id | channel | event_type | timestamp
    123        | CHAT    | CREATED    | 10:10:01
    123        | CHAT    | CREATED    | 07:56:45 (first occurrence)
    123        | CHAT    | CREATED    | 08:13:33
```

- After cleaning the data we're able to create the busines report

In [43]:
from pyspark.sql.window import Window
from pyspark.sql.functions import *

window = Window.partitionBy("message_id", "event_type", "channel").orderBy("timestamp")
df_valid_with_rownum = df_valid.withColumn("row_num", row_number().over(window))
df_valid_deduped = df_valid_with_rownum.filter(col("row_num") == 1).drop("row_num")

### Report 1
  - Aggregate data by date, event_type and channel
  - Count number of messages
  - pivot event_type from rows into columns
  - schema expected:
  
```
|      date|channel|CLICKED|CREATED|OPEN|RECEIVED|SENT|
+----------+-------+-------+-------+----+--------+----+
|2024-12-03|    SMS|      4|      4|   1|       1|   5|
|2024-12-03|   CHAT|      3|      7|   5|       8|   4|
|2024-12-03|   PUSH|   NULL|      3|   4|       3|   4|
```

In [45]:
# report 1
# TODO

pivot_df = df_valid_deduped.groupBy("date", "channel") \
    .pivot("event_type") \
    .agg(count("*"))

pivot_df.show()

+----------+-------+-------+-------+----+--------+----+
|      date|channel|CLICKED|CREATED|OPEN|RECEIVED|SENT|
+----------+-------+-------+-------+----+--------+----+
|2025-07-16|  EMAIL|      3|      3|   2|       1|   2|
|2025-07-16|    SMS|      4|      3|   2|       6|   5|
|2025-07-16|   CHAT|      1|      4|   3|       2|   2|
|2025-07-16|  OTHER|      2|      4|   1|       2|   2|
|2025-07-16|   PUSH|      1|      3|   2|       1|   4|
+----------+-------+-------+-------+----+--------+----+



## Report 2

- Identify the most active users by channel (sorted by number of iterations)
- schema expected:

```
+-------+----------+----+-----+-----+----+---+
|user_id|iterations|CHAT|EMAIL|OTHER|PUSH|SMS|
+-------+----------+----+-----+-----+----+---+
|   1022|         5|   2|    0|    1|   0|  2|
|   1004|         4|   1|    1|    1|   1|  0|
|   1013|         4|   0|    0|    2|   1|  1|
|   1020|         4|   2|    0|    1|   1|  0|
```


In [60]:
# report 2
# TODO
from pyspark.sql.functions import count, coalesce, lit, sum as spark_sum
from functools import reduce

pivot_df = df_valid_deduped.groupBy("user_id") \
    .pivot("channel") \
    .agg(count("*"))

# fill nulls with 0
pivot_df = pivot_df.fillna(0)
#pivot_df.show()

# add iterarions column
# Calculate total number of iterations across channels
channel_cols = [c for c in pivot_df.columns if c != "user_id"]
active_users_by_channel_df = pivot_df.withColumn("iterations", reduce(lambda x, y: x + y, [col(c) for c in channel_cols]))
#active_users_by_channel_df = pivot_df.withColumn("iterations", sum(pivot_df[col] for col in pivot_df.columns if col != "user_id"))

active_users_by_channel_df.show()

+-------+----+-----+-----+----+---+----------+
|user_id|CHAT|EMAIL|OTHER|PUSH|SMS|iterations|
+-------+----+-----+-----+----+---+----------+
|   1016|   0|    1|    0|   0|  0|         1|
|   1005|   0|    0|    1|   0|  0|         1|
|   1034|   0|    0|    0|   0|  1|         1|
|   1030|   2|    1|    0|   0|  1|         4|
|   1046|   0|    0|    0|   1|  1|         2|
|   1008|   0|    0|    0|   0|  1|         1|
|   1047|   0|    0|    0|   1|  0|         1|
|   1021|   0|    0|    1|   1|  1|         3|
|   1026|   0|    0|    1|   0|  1|         2|
|   1028|   0|    1|    0|   0|  0|         1|
|   1029|   0|    0|    0|   0|  1|         1|
|   1032|   0|    1|    0|   0|  0|         1|
|   1010|   0|    0|    0|   0|  1|         1|
|   1050|   0|    0|    0|   0|  1|         1|
|   1002|   0|    0|    1|   0|  1|         2|
|   1048|   0|    1|    0|   0|  0|         1|
|   1035|   0|    0|    0|   0|  1|         1|
|   1017|   0|    0|    0|   1|  0|         1|
|   1037|   0

# Challenge 3

In [ ]:
# Theoretical question:

# A new usecase requires the message data to be aggregate in near real time
# They want to build a dashboard embedded in the platform website to analyze message data in low latency (few minutes)
# This application will access directly the data aggregated by streaming process

# Q1:
- What would be your suggestion to achieve that using Spark Structure Streaming?
Or would you choose a different data processing tool?

# Q2:
- Which storage would you use and why? (database?, data lake?, kafka?)

